In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#это мне всё, конечно же, понадобится

In [ ]:
y_train = pd.read_parquet('/kaggle/input/y-train/y_train.parquet', engine = 'auto')

In [ ]:
y_train

In [ ]:
import re

instances = set()
parts = set()
for col in y_train.columns:
    instances.add(col.split("_")[1])
    replacement = re.sub(r"ЭКСГАУСТЕРА №\d+","ЭКСГАУСТЕРА №d",col.split("_")[2])
    replacement = re.sub(r"ЭКСГ.\d+", "ЭКСГ.d", replacement)
    replacement = re.sub(r"ЭКСГ\d+","ЭКСГ.d", replacement)
    replacement = re.sub(r"ЭКСГ№\d+","ЭКСГ.d", replacement)
    replacement = re.sub(r"ЭКСГ.№\d+","ЭКСГ.d", replacement)
    replacement = re.sub(r"ЭКСГ. №\d+","ЭКСГ.d", replacement)
    replacement = re.sub(r"ЭКСГАУСТЕРА А/М №\d+","ЭКСГ.d", replacement)
    replacement = re.sub("МАСЛОПРОВОДЫ ЭКСГ №d","МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №d",replacement)
    replacement = re.sub("ЭЛЕКТРОАППАРАТУРА ЭКСГ.d", "ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №d",replacement)
    replacement = re.sub("ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГ.d", "ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №d",replacement)
    replacement = re.sub("ЭЛЕКТРОАППАРАТУРА ЭКСГ. №d", "ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №d",replacement)
    replacement = re.sub("ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГ №d", "ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №d",replacement).strip()
    replacement = re.sub(r"ЭКСГАУСТЕР А/М №\d+","ЭКСГАУСТЕР №d", replacement)
    replacement = re.sub(r"МАСЛОПРОВОДЫ ЭКСГ.d","МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №d",replacement)
    if replacement == "ПОДШИПНИК ОПОРНЫЙ №1":
        replacement = "ПОДШИПНИК ОПОРНЫЙ №1 ЭКСГ.d"
    if replacement == "ПОДШИПНИК ОПОРНЫЙ №2":
        replacement = "ПОДШИПНИК ОПОРНЫЙ №2 ЭКСГ.d"
    parts.add(replacement)
matrix = pd.DataFrame(sorted(parts))
matrix = matrix.set_index(0)
for col in instances:
    matrix[col] = 0
for col in y_train.columns:
    instances.add(col.split("_")[1])
    replacement = re.sub(r"ЭКСГАУСТЕРА №\d+","ЭКСГАУСТЕРА №d",col.split("_")[2])
    replacement = re.sub(r"ЭКСГ.\d+", "ЭКСГ.d", replacement)
    replacement = re.sub(r"ЭКСГ\d+","ЭКСГ.d", replacement)
    replacement = re.sub(r"ЭКСГ№\d+","ЭКСГ.d", replacement)
    replacement = re.sub(r"ЭКСГ.№\d+","ЭКСГ.d", replacement)
    replacement = re.sub(r"ЭКСГ. №\d+","ЭКСГ.d", replacement)
    replacement = re.sub(r"ЭКСГАУСТЕРА А/М №\d+","ЭКСГ.d", replacement)
    replacement = re.sub(r"ЭКСГАУСТЕР А/М №\d+","ЭКСГАУСТЕР №d", replacement)
    replacement = re.sub("МАСЛОПРОВОДЫ ЭКСГ №d","МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №d",replacement)
    replacement = re.sub("ЭЛЕКТРОАППАРАТУРА ЭКСГ.d", "ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №d",replacement)
    replacement = re.sub("ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГ.d", "ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №d",replacement)
    replacement = re.sub("ЭЛЕКТРОАППАРАТУРА ЭКСГ. №d", "ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №d",replacement)
    replacement = re.sub("ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГ №d", "ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №d",replacement).strip()
    replacement = re.sub(r"МАСЛОПРОВОДЫ ЭКСГ.d","МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №d",replacement)
    if replacement == "ПОДШИПНИК ОПОРНЫЙ №1":
        replacement = "ПОДШИПНИК ОПОРНЫЙ №1 ЭКСГ.d"
    if replacement == "ПОДШИПНИК ОПОРНЫЙ №2":
        replacement = "ПОДШИПНИК ОПОРНЫЙ №2 ЭКСГ.d"
    if matrix.loc[replacement,col.split("_")[1]] == 0:
        matrix.loc[replacement,col.split("_")[1]] = [col]
    else:
        matrix.loc[replacement,col.split("_")[1]].append(col)

def zero_to_none(x):
    if x == 0:
        return None
    else:
        return x
    
def listify(x):
    if type(x) != list and not (x is None):
        return [x]
    else:
        return x
    
matrix = matrix.applymap(zero_to_none)
matrix = matrix.applymap(listify)
matrix

In [ ]:
def collect(array):
    answer = []
    for a in array:
        if not (a is None):
            for it in a:
                answer.append(it)
    if len(answer)== 0:
        answer = None
    return answer


matrix["total"] = matrix.apply(lambda x: collect(x), axis=1) 
matrix["total"].to_csv("dict_long.csv")

In [ ]:
for part in matrix["total"].index.values:
    filt = matrix["total"].loc[part]
    print(part)
    y_train[filt].to_csv("y_long_"+part.replace("/","")+".csv")